# Numerical Filtering
This notebook contains the numeric computations and the implementatiosn of the Mueller filters of (Zander et al 2020).
We start with loading the necessary packages.
For the correctness of the transformation `T` which maps Mueller matrices to its corresponding coherency matrices  see the "Symbolic Notebook". 

In [1]:
import numpy as np
from numpy import linalg as LA

We set up the transformation of the Mueller matrix to the Hermitian coherency matrix and vice versa.

In [2]:
I=1.j
Tn=1/2*np.array([[1,  0,  0,  0,    0,  1,  0,  0,    0,  0,  1,  0,     0,  0,  0,  1],# first row
                 [0,  1,  0,  0,    1,  0,  0,  0,    0,  0,  0, -I,     0,  0,  I,  0],
                 [0,  0,  1,  0,    0,  0,  0,  I,    1,  0,  0,  0,     0, -I,  0,  0],
                 [0,  0,  0,  1,    0,  0, -I,  0,    0,  I,  0,  0,     1,  0,  0,  0],
                 [0,  1,  0,  0,    1,  0,  0,  0,    0,  0,  0,  I,     0,  0, -I,  0],# second row
                 [1,  0,  0,  0,    0,  1,  0,  0,    0,  0, -1,  0,     0,  0,  0, -1],
                 [0,  0,  0,  I,    0,  0,  1,  0,    0,  1,  0,  0,    -I,  0,  0,  0],
                 [0,  0,  I,  0,    0,  0,  0,  1,   -I,  0,  0,  0,     0,  1,  0,  0],
                 [0,  0,  1,  0,    0,  0,  0, -I,    1,  0,  0,  0,     0,  I,  0,  0],# third row
                 [0,  0,  0, -I,    0,  0,  1,  0,    0,  1,  0,  0,     I,  0,  0,  0],
                 [1,  0,  0,  0,    0, -1,  0,  0,    0,  0,  1,  0,     0,  0,  0, -1],
                 [0,  I,  0,  0,   -I,  0,  0,  0,    0,  0,  0,  1,     0,  0,  1,  0],
                 [0,  0,  0,  1,    0,  0,  I,  0,    0, -I,  0,  0,     1,  0,  0,  0],# fourth row
                 [0,  0, -I,  0,    0,  0,  0,  1,    I,  0,  0,  0,     0,  1,  0,  0],
                 [0, -I,  0,  0,    I,  0,  0,  0,    0,  0,  0,  1,     0,  0,  1,  0],
                 [1,  0,  0,  0,    0, -1,  0,  0,    0,  0, -1,  0,     0,  0,  0,  1]],
                dtype=complex)
Tinv = LA.inv(Tn)
def T(x):
    return np.reshape((Tn @ np.reshape(x, (16,1))), (4,4))
def invT(x):
    return np.reshape((Tinv @ np.reshape(x, (16,1))), (4,4))

We are now ready to setup the filter of the measured Mueller matrix.
First the generic filter which filters the negative eigenvalues by setting them to zero.
This is the filter proposed in *(Cloude 1990)*.

In [3]:
def GenericFilter(x):
    e,v =LA.eigh(T(x))
    D=np.diag(np.clip(e,0,None))
    return (invT(v @ D @ LA.inv(v))).real

If we assumed that our measured Mueller matrices M can be decomposed as a sum of a non-depolarising part and a perfectly depolarising Mueller matrix then we have to define the filter as follows.
Note, the second function returns the pair of matrices instead of the sum of them.

In [4]:
def NonPerfectFilter(x):
    e,v =LA.eigh(T(x))
    d=(e[0]+e[1]+e[2])/3
    D=np.diag(np.array([e[3],d,d,d]))
    return (invT(v @ D @ LA.inv(v))).real
def NonPerfectFilterPair(x):
    e,v =LA.eigh(T(x))
    d=(e[0]+e[1]+e[2])/3
    D1=np.diag(np.array([d,d,d,d]))
    D2=np.diag(np.array([e3-d,0,0,0]))
    return (invT(v @ D1 @ LA.inv(v))).real, (invT(v @ D1 @ LA.inv(v))).real; 

Now we want to the preserve the upper left entry $M_{00}$ of a measured Mueller matrix $M$ after filterning,
e.g. the change in intensity of an unpolarised beam is believed to be measured correctly.
For this it is enough to preserve the value of the sum of all eigenvalues of the coherency matrix. We also plan to implement this filter in a way, such that for any given value $c$ we can set the filter to produce the nearest Mueller matrix with $M_{00}=c$. 

In [5]:
def GenericM00eigenFilter(y):
    neg_e=y[y <= 0]
    d=0
    d=np.sum(neg_e)/(4-np.size(neg_e))
    y=np.clip(y,0,None)
    y[y != 0]=y[y != 0]+d
    # check for newly produced negative
    # eigenvalues and filter again if necessary
    neg_y=np.clip(y,None,0)
    s=np.size((neg_y[neg_y != 0]))
    if (s != 0):
        y=GenericM00eigenFilter(y)
    # done
    return y 
def GenericZfilter(x):
    e,v =LA.eigh(T(x))
    D=GenericM00eigenFilter(e)
    D=np.diag(D)
    return (invT(v @ D @ LA.inv(v))).real

Now take some measured Mueller matrices.
The matrices `M1`, `M2` and `M3` are the examples used in *(Cloude 1990)*.
The matrices `M4` and `M4filt` are the examples from *(Aiello et al. 2006)*, where `M4filt` is filtered matrix which was obtained in the paper.

In [6]:
M1=np.array([[0.997 ,  0.003 , 0.017, 0.035 ],
             [ 0.002 ,  1.000,-0.03, -0.0010],
             [0.076, 0.022, 0.091, -1.013],
             [0.010, 0.034, 0.976, 0.232]])
M2=np.array([[0.8488, -0.0503, 0.0294, 0.0617 ],
             [-0.0503,  0.8403, 0.0913, -0.0920],
             [0.0294, 0.0913, 0.8277, 0.000],
             [0.0617, -0.0920, 0.0, 0.7947]])
M3=np.array([[0.8886, -0.0131, 0.0055, 0.0786 ],
             [-0.0115,  0.5762,-0.2820, -0.1668],
             [0.0048,-0.2809, 0.6825, 0.0026],
             [0.0775, -0.1672, 0.0012, 0.8061]])
M4=np.array([[0.7599, 0.0295, 0.1185, -0.0623],
             [0.0384, 0.5394, 0.0282, -0.1714],
             [0.1240, -0.012, 0.6608, 0.2168],
             [-0.0573, -0.1811, -0.1863, 0.4687]])
M4filt=np.array([[0.7599, 0.0257, 0.1206, -0.0576],
                 [0.0372, 0.5285, 0.0001, -0.0496],
                 [0.1208, -0.0001, 0.6184, 0.1920],
                 [-0.0554, -0.0572, -0.1794, 0.4822]])

We might wonder how much the optimal filtering,
which preserves the upper left element of the measured Mueller matrix,
differs from the naive filtering
where one first filters the negative eigenvalues and then scales the filtered Mueller matrix. 
Moreover, as we compare the given filtered `M4filt` to our optimal method.  

In [7]:
print(LA.norm(GenericZfilter(M1)-M1)/LA.norm(GenericFilter(M1)/GenericFilter(M1)[0,0]*M1[0,0]-M1))
print(LA.norm(GenericZfilter(M2)-M2)/LA.norm(GenericFilter(M2)/GenericFilter(M2)[0,0]*M2[0,0]-M2))
print(LA.norm(GenericZfilter(M3)-M3)/LA.norm(GenericFilter(M3)/GenericFilter(M3)[0,0]*M3[0,0]-M3))
print(LA.norm(GenericZfilter(M4)-M4)/LA.norm(GenericFilter(M4)/GenericFilter(M4)[0,0]*M4[0,0]-M4))
print(LA.norm(GenericZfilter(M4)-M4)/LA.norm(M4filt-M4))
print(GenericZfilter(M1))
print(GenericZfilter(M2))
print(GenericZfilter(M3))
print(GenericZfilter(M4))

0.8601209863914068
0.896149906656278
0.923253443137845
0.9110202764866298
0.9264795025656251
[[ 0.997       0.00340004 -0.00720393  0.00827384]
 [ 0.00128542  0.96681889 -0.01724335  0.01136879]
 [ 0.04737567  0.03025474  0.14399661 -0.97246578]
 [ 0.03984697  0.02652134  0.96374819  0.17157183]]
[[ 0.8488     -0.04939662  0.02736674  0.0606842 ]
 [-0.04939662  0.80377171  0.03584098 -0.03015221]
 [ 0.02736674  0.03584098  0.77845166 -0.01939577]
 [ 0.0606842  -0.03015221 -0.01939577  0.76952802]]
[[ 0.8886     -0.01507501  0.00394991  0.07736545]
 [-0.01399667  0.61205675 -0.11515513 -0.09260091]
 [ 0.00298897 -0.11399846  0.62181666  0.05834267]
 [ 0.07723833 -0.09285789  0.05711989  0.64482085]]
[[ 0.7599      0.02729742  0.11326012 -0.05396825]
 [ 0.04591394  0.52869671  0.00502863 -0.0743151 ]
 [ 0.12723247 -0.00252733  0.56564671  0.19425018]
 [-0.05819589 -0.08607419 -0.1796437   0.48118747]]


Hence in this case the optimal filtering in our examples get us 7 to 14 percent improvements. 

## References

Aiello, A, G Puentes, D Voigt, and JP Woerdman. 2006. “Maximum-Likelihood Estimation of Mueller Matrices.” Optics Letters 31 (6): 817–19.

Cloude, Shane R. 1990. “Conditions For The Physical Realisability Of Matrix Operators In Polarimetry.” In Polarization Considerations for Optical Systems II, edited by Russell A. Chipman, 1166:177–87. SPIE. doi:10.1117/12.962889.

Zander, T and J Beyerer. 2020. “The Mueller matrix cone and its application to filtering.” to appear
